In [45]:
tmp = np.array([[1, 2, 3, 4], [5, 4, 7, 8]])
tmp_derivative = tmp.copy()
tmp_max = np.max(tmp, axis=1)
for i in range(tmp.shape[1]):
    tmp_derivative[:, i] = tmp[:, i] - tmp_max 
tmp_derivative = np.exp(tmp_derivative)
print(tmp_derivative)
tmp_s = np.sum(tmp_derivative, axis=1)
print(tmp_s)

tmp_hat = tmp_derivative.copy()
for i in range(tmp.shape[0]):
    tmp_hat[i, :] = tmp_hat[i, :] / tmp_s[i]
print(tmp_hat)

[[0.04978707 0.13533528 0.36787944 1.        ]
 [0.04978707 0.01831564 0.36787944 1.        ]]
[1.55300179 1.43598215]
[[0.0320586  0.08714432 0.23688282 0.64391426]
 [0.03467109 0.01275478 0.25618664 0.69638749]]


In [47]:
tmp_a = np.array([[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]])
tmp_b = np.array([[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]])
tmp_a * tmp_b

array([[ 1,  4,  9, 16],
       [ 1,  4,  9, 16],
       [ 1,  4,  9, 16],
       [ 1,  4,  9, 16]])

In [78]:
tmp_a = np.array([[0 ,0, 0, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0]])
tmp_index = []
for row in tmp_a:
    tmp_index.extend([element_index for element_index in range(len(row)) if row[element_index] == 1])
tmp_index = [element_index for element_index in range(len(tmp_index)) if tmp_index[element_index]==1]
tmp_index

[1]

In [189]:
tmp_a = np.array([[0.5 ,0.5, 0, 0.75, 0.3], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0]])

for row in tmp_a:
    print(np.argmax(row))

3
1
0


In [145]:
import numpy as np
import matplotlib.pyplot as plt
from util import get_mnist_data
from logistic_np import add_one, LogisticClassifier
from sklearn.metrics import confusion_matrix

In [167]:
train_x, train_y, val_x, val_y, test_x, test_y = get_mnist_data()

Reading fashion MNIST data...
Done reading


In [168]:
class SoftmaxClassifier(LogisticClassifier):
    def __init__(self, w_shape):
        super(SoftmaxClassifier, self).__init__(w_shape)
    def feed_forward(self, x):
        z = np.dot(x, self.w)
        z_derivative = z.copy()
        z_max = np.max(z, axis=1)
        
        for i in range(z.shape[1]):
            z_derivative[:, i] = z[:, i] - z_max
        z_derivative = np.exp(z_derivative)
        
        s = np.sum(z_derivative, axis=1)
        
        y_hat = z_derivative.copy()
        for i in range(y_hat.shape[0]):
            y_hat[i, :] = y_hat[i, :] / s[i]
        return y_hat
    
    def compute_loss(self, y, y_hat):
        return -np.sum(y * np.log(y_hat))/float(y.shape[0])
    
    def get_grad(self, x, y, y_hat):
        return np.dot(x.T, (y_hat - y))/float(y.shape[0])

In [169]:
def normalize(train_x, test_x, val_x):
    mean = np.sum(train_x)/float(train_x.shape[0] * train_x.shape[1])
    std = np.sqrt(np.sum(np.square(train_x - mean))/float(train_x.shape[0] * train_x.shape[1]))

    train_x = (train_x - mean)/float(std)
    test_x = (test_x - mean)/float(std)
    val_x = (val_x - mean)/float(std)
    return train_x, val_x, test_x

In [170]:
def create_one_hot(labels, num_k=10):
    one_hot_label = []
    for label in labels:
        tmp = [0] * num_k
        tmp[label] = 1
        one_hot_label.append(tmp)
    
    return np.array(one_hot_label)

In [171]:
def test(y_hat, test_y):
    confusion_mat_1 = np.zeros((10,10), dtype=np.float64)
    
    test_y_without_onehot = []
    y_hat_without_onehot = []
    
    for row in test_y:
        test_y_without_onehot.append(np.argmax(row))
    
    for row in y_hat:
        y_hat_without_onehot.append(np.argmax(row))
     
    occur = np.bincount(test_y_without_onehot)
    # print(occur)
    # print(len(test_y_without_onehot))
#     print(y_hat_without_onehot)
    
    confusion_mat = confusion_matrix(test_y_without_onehot, y_hat_without_onehot)
    #print(confusion_mat)
    #print(occur)
    for row_index in range(confusion_mat.shape[0]):
        confusion_mat_1[row_index, :] = confusion_mat[row_index, :]/float(occur[row_index])
        
    print(confusion_mat_1)
#     for i in range(10):
#         test_y_i = test_y_without_onehot[element_index for element_index in range(len(test_y_without_onehot)) \
#                                          if test_y_without_onehot[element_index]==i]
        

In [172]:
a = -10e-3
abs(a)

0.01

In [173]:
train_y = create_one_hot(train_y)
val_y = create_one_hot(val_y)
test_y = create_one_hot(test_y)

train_x, val_x, test_x = normalize(train_x, val_x, test_x)

num_feature = train_x.shape[1]
dec_classifier = SoftmaxClassifier((num_feature, 10))
momentum = np.zeros_like(dec_classifier.w)

num_epoch = 100
learning_rate = 0.01
momentum_rate = 0.9
epochs_to_draw = 10
all_train_loss = []
all_val_loss = []

for e in range(num_epoch):    
        train_y_hat = dec_classifier.feed_forward(train_x)
        val_y_hat = dec_classifier.feed_forward(val_x)

        train_loss = dec_classifier.compute_loss(train_y, train_y_hat)
        val_loss = dec_classifier.compute_loss(val_y, val_y_hat)

        grad = dec_classifier.get_grad(train_x, train_y, train_y_hat)
        
        dec_classifier.update_weight(grad, learning_rate)
        
        all_train_loss.append(train_loss) 
        all_val_loss.append(val_loss)
        
        if e % 10 == 0:
            print("Epoch %d: train loss: %.5f || val loss: %.5f" % (e+1, train_loss, val_loss))

test_y_hat = dec_classifier.feed_forward(train_x)
test(test_y_hat, train_y)

Epoch 1: train loss: 2.93782 || val loss: 3.07679
Epoch 11: train loss: 1.51064 || val loss: 3.06391
Epoch 21: train loss: 1.16940 || val loss: 3.42843
Epoch 31: train loss: 1.01900 || val loss: 3.72563
Epoch 41: train loss: 0.93101 || val loss: 3.95923
Epoch 51: train loss: 0.87158 || val loss: 4.14950
Epoch 61: train loss: 0.82790 || val loss: 4.30955
Epoch 71: train loss: 0.79395 || val loss: 4.44755
Epoch 81: train loss: 0.76648 || val loss: 4.56882
Epoch 91: train loss: 0.74359 || val loss: 4.67698
[[0.80152672 0.01526718 0.01526718 0.07633588 0.00763359 0.01908397
  0.05725191 0.         0.00763359 0.        ]
 [0.0078125  0.91796875 0.00390625 0.046875   0.00390625 0.00390625
  0.015625   0.         0.         0.        ]
 [0.00816327 0.00408163 0.64897959 0.         0.15510204 0.04897959
  0.10612245 0.         0.02857143 0.        ]
 [0.05063291 0.05063291 0.00843882 0.82700422 0.01687764 0.00421941
  0.03375527 0.         0.00843882 0.        ]
 [0.         0.02352941 0.13333

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from util import get_mnist_data
from logistic_np import add_one
from softmax_np import *

In [5]:
train_x, train_y, val_x, val_y, test_x, test_y = get_mnist_data()

train_y = create_one_hot(train_y)
val_y = create_one_hot(val_y)
test_y = create_one_hot(test_y)

train_x, val_x, test_x = normalize(train_x, val_x, test_x)
    
train_x = add_one(train_x) 
val_x = add_one(val_x)
test_x = add_one(test_x)

x = tf.placeholder(tf.float64, name='x')
y = tf.placeholder(tf.float64, name='y')

w_shape = (train_x.shape[1], 10)
w = tf.Variable(np.random.normal(0, np.sqrt(2.0/np.sum(w_shape)), w_shape), name='weights')

numerator = tf.exp(tf.matmul(x, w))
denominator = tf.reduce_sum(numerator, 1, keepdims=True)
pred = tf.divide(numerator, denominator)

cost = -tf.reduce_mean(y * tf.log(pred))

num_epoch = 1000
learning_rate = 0.01

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

epochs_to_draw = 1
all_train_loss = []
all_val_loss = []

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for e in range(num_epoch):
        sess.run(optimizer, feed_dict = {x: train_x, y: train_y})
        # [TODO 1.16] Compute loss and update weights here
        loss = sess.run(cost, feed_dict = {x: train_x, y: train_y})
        # Update weights...
        
        if e % 100 == 0:
            print("Epoch %d: loss is %.5f" % (e+1, loss))
        
    y_hat = sess.run(pred, feed_dict={x: test_x})
    test(y_hat, test_y)

Reading fashion MNIST data...
Done reading
Epoch 1: loss is 0.30646
Epoch 101: loss is 0.14849
Epoch 201: loss is 0.11653
Epoch 301: loss is 0.10238
Epoch 401: loss is 0.09403
Epoch 501: loss is 0.08834
Epoch 601: loss is 0.08411
Epoch 701: loss is 0.08079
Epoch 801: loss is 0.07808
Epoch 901: loss is 0.07580
Confusion matrix:
[[0.85 0.04 0.02 0.04 0.02 0.   0.04 0.   0.   0.  ]
 [0.   0.93 0.   0.07 0.   0.   0.   0.   0.   0.  ]
 [0.04 0.   0.43 0.04 0.26 0.   0.22 0.   0.   0.  ]
 [0.05 0.03 0.03 0.72 0.07 0.   0.1  0.   0.   0.  ]
 [0.   0.   0.12 0.09 0.65 0.02 0.09 0.   0.02 0.  ]
 [0.   0.   0.   0.   0.   0.83 0.   0.11 0.02 0.04]
 [0.36 0.   0.09 0.02 0.3  0.02 0.19 0.   0.02 0.  ]
 [0.   0.   0.   0.   0.   0.1  0.   0.78 0.   0.12]
 [0.04 0.   0.   0.02 0.   0.04 0.   0.04 0.87 0.  ]
 [0.   0.   0.   0.   0.   0.08 0.   0.02 0.   0.9 ]]
Diagonal values:
[0.85 0.93 0.43 0.72 0.65 0.83 0.19 0.78 0.87 0.9 ]


In [3]:
np.sum(y_3, axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

In [9]:
?tf.divide

Signature: tf.divide(x, y, name=None)
Docstring: Computes Python style division of `x` by `y`.
File:      d:\anaconda\envs\tsf36\lib\site-packages\tensorflow\python\ops\math_ops.py
Type:      function
